In [70]:
import glob
import os

In [72]:
dataset_path = "./asl_alphabet_data"

In [128]:
#grabs all the folder names in the dataset and sorts them
all_classes = sorted([os.path.basename(folder) for folder in glob.glob(os.path.join(dataset_path, "*"))])

def print_classes(title, class_list, rows, cols):
    print(f"{title}: {len(class_list)}")
    print("\nDataset Classes:")
    index = 0
    for row in range(rows):
        row_classes = []
        for col in range(cols):
            row_classes.append(class_list[index])
            index += 1
        print(" | ".join(row_classes))

print_classes("Total Dataset Classes", all_classes, 3, 9)

Total Dataset Classes: 27

Dataset Classes:
A | B | C | D | E | F | G | H | I
J | K | L | M | N | O | P | Q | R
S | T | U | V | W | X | Y | Z | nothing


---
###Selected Letters 

I'm picking a smaller set of ASL letters to make the dataset easier to work with and keep things running smoothly on my computer. Some letters, like "J" and "Z," need movement to sign, so they don’t work as well with static images.

In [131]:
#list of ASL letters we selected for this project
asl_keep = ["A", "B", "C", "D", "E", "F", "H", "I", "L", "O", "Q", "R", "S", "U", "V", "W", "X", "Y"]

print_classes("Selected ASL Letters", asl_keep, 2, 9)

Selected ASL Letters: 18

Dataset Classes:
A | B | C | D | E | F | H | I | L
O | Q | R | S | U | V | W | X | Y
